In [1]:
import os
import importlib
os.chdir('/ictstr01/home/icb/fatemehs.hashemig/codes/interpretable-ssl')

# import

In [2]:
import interpretable_ssl.trainers.scpoli_original
importlib.reload(interpretable_ssl.trainers.scpoli_original)

import experiment_evaluator
importlib.reload(experiment_evaluator)

import interpretable_ssl.models.linear 
importlib.reload(interpretable_ssl.models.linear)
from interpretable_ssl.models.linear import *

INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
 captum (see https://github.com/pytorch/captum).


In [3]:
from experiment_evaluator import *

In [4]:
from interpretable_ssl.trainers.swav import *



init trainer
load the model
encode the ref, and query

train linear classifier
train mlp
knn, kmeans


In [5]:
from interpretable_ssl.datasets.immune import *

In [6]:
ds = ImmuneDataset()

In [7]:
e = ExperimentEvaluator(ds)

# init trainer, encode, ref and query

In [25]:
t = SwAV(augmentation_type = 'community', dimensionality_reduction='pca', k_neighbors=7, debug=True, num_prototypes=300, batch_size=1024,
        prot_decoding_loss_scaler=0, cvae_loss_scaler=0.0, training_type='semi_supervised', epsilon=0.02, model_version=1)
t.setup()

dataset is None, loading pbmc-immune


INFO - 10/16/24 12:22:17 - 0:00:00 - ============ Initialized logger ============
INFO - 10/16/24 12:22:17 - 0:00:00 - all_latent: None
                                     augmentation_type: community
                                     base_lr: 4.8
                                     batch_size: 1024
                                     cell_type_key: cell_type
                                     checkpoint_freq: 25
                                     condition_key: study
                                     crops_for_assign: [0, 1]
                                     cvae_loss_scaler: 0.0
                                     cvae_reg: 0
                                     dataset: pbmc-immune
                                     dataset_id: pbmc-immune
                                     debug: True
                                     default_values: {'dataset_id': 'pbmc-immune', 'model_name_version': 3, 'num_prototypes': 128, 'hidden_dim': 64, 'latent_dims': 8, 'batch_size'

Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



In [26]:
t.model = None
encoded_ref = t.encode_ref()
encoded_query = t.encode_query()

100%|██████████| 29/29 [00:17<00:00,  1.69it/s]


# train linear classifier

In [46]:
# Usage
# Assuming `embeddings` is your tensor of shape (n_samples, n_features)
# and `df['cell_type']` contains your labels.

def evaluate(embedding, obs):
    classifier = LinearClassifier(embedding, obs['cell_type'], batch_size=64, epochs=20)
    classifier.train()
    
    # Get F1 scores as DataFrame
    f1_scores_df = classifier.evaluate()
    return f1_scores_df

In [44]:
evaluate(encoded_ref, t.ref.adata.obs)

,Class,Train F1 Score,Test F1 Score
0,CD10+ B cells,0.782288,0.857143
1,CD14+ Monocytes,0.938653,0.936561
2,CD16+ Monocytes,0.911672,0.917431
3,CD20+ B cells,0.979242,0.972921
4,CD4+ T cells,0.883417,0.883688
5,CD8+ T cells,0.002602,0.000000
6,Erythrocytes,0.893825,0.872727
7,Erythroid progenitors,0.824632,0.757895
8,HSPCs,0.628218,0.649057
9,Megakaryocyte progenitors,0.088106,0.074074


In [48]:
evaluate(encoded_query, t.query.adata.obs)

Epoch 1/20: 100%|██████████| 55/55 [00:00<00:00, 117.08batch/s, loss=2.3389]


Epoch 1/20 completed. Average Loss: 2.3389


Epoch 2/20: 100%|██████████| 55/55 [00:00<00:00, 638.89batch/s, loss=2.2370]


Epoch 2/20 completed. Average Loss: 2.2370


Epoch 3/20: 100%|██████████| 55/55 [00:00<00:00, 613.61batch/s, loss=2.1431]


...

Epoch 19/20: 100%|██████████| 55/55 [00:00<00:00, 642.47batch/s, loss=1.3048]


Epoch 19/20 completed. Average Loss: 1.3048


Epoch 20/20: 100%|██████████| 55/55 [00:00<00:00, 645.06batch/s, loss=1.2750]


Epoch 20/20 completed. Average Loss: 1.2750


,Class,Train F1 Score,Test F1 Score
0,CD14+ Monocytes,0.733087,0.726316
1,CD16+ Monocytes,0.000000,0.000000
2,CD20+ B cells,0.945568,0.959064
3,CD4+ T cells,0.774797,0.769231
4,CD8+ T cells,0.000000,0.000000
5,Megakaryocyte progenitors,0.000000,0.000000
6,Monocyte-derived dendritic cells,0.735343,0.689655
7,NK cells,0.798906,0.773006
8,NKT cells,0.629213,0.654762
9,Plasmacytoid dendritic cells,0.066225,0.054054


# init scpoli, encode data

do the same for scpoli encoded data
+ classifiy using scpoli classification method

In [74]:
t_dict = {'scpoli': {
            'latent_dims': [8],
            'batch_size': [1024],
            'experiment_name': ['scpoli-semi'],
            # 'dataset_id':['hlca'],
            'model_name_version': [3],
            }}

In [75]:
trainers = e.generate_trainers(t_dict)

Generating trainers for scpoli: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s, Memory=5.4%, CPU=1.9%, trainer_name=scpoli_exp_scpoli-semi]


In [76]:
st = trainers[0]

In [78]:
st.model = None
ref_emb = st.encode_ref()
query_emb = st.encode_query()

Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



100%|██████████| 29/29 [00:05<00:00,  5.29it/s]


Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 

Embedding dictionary:
 	Num conditions: [5]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



100%|██████████| 5/5 [00:00<00:00, 12.79it/s]


In [79]:
evaluate(ref_emb, st.ref.adata.obs)

Epoch 1/20: 100%|██████████| 365/365 [00:01<00:00, 208.01batch/s, loss=2.4363]


Epoch 1/20 completed. Average Loss: 2.4363


Epoch 2/20: 100%|██████████| 365/365 [00:00<00:00, 623.16batch/s, loss=2.0364]


Epoch 2/20 completed. Average Loss: 2.0364


Epoch 3/20: 100%|██████████| 365/365 [00:00<00:00, 624.01batch/s, loss=1.8372]


...

Epoch 19/20: 100%|██████████| 365/365 [00:00<00:00, 646.46batch/s, loss=0.9515]


Epoch 19/20 completed. Average Loss: 0.9515


Epoch 20/20: 100%|██████████| 365/365 [00:00<00:00, 626.97batch/s, loss=0.9310]


Epoch 20/20 completed. Average Loss: 0.9310


,Class,Train F1 Score,Test F1 Score
0,CD10+ B cells,0.000000,0.000000
1,CD14+ Monocytes,0.904941,0.903824
2,CD16+ Monocytes,0.000000,0.000000
3,CD20+ B cells,0.880851,0.885965
4,CD4+ T cells,0.861333,0.861255
5,CD8+ T cells,0.000000,0.000000
6,Erythrocytes,0.599471,0.558499
7,Erythroid progenitors,0.000000,0.000000
8,HSPCs,0.000000,0.000000
9,Megakaryocyte progenitors,0.000000,0.000000


In [80]:
evaluate(query_emb, st.query.adata.obs)

Epoch 1/20: 100%|██████████| 55/55 [00:00<00:00, 599.07batch/s, loss=2.2025]


Epoch 1/20 completed. Average Loss: 2.2025


Epoch 2/20: 100%|██████████| 55/55 [00:00<00:00, 645.64batch/s, loss=2.1413]


Epoch 2/20 completed. Average Loss: 2.1413


Epoch 3/20: 100%|██████████| 55/55 [00:00<00:00, 615.51batch/s, loss=2.0902]


...

Epoch 19/20: 100%|██████████| 55/55 [00:00<00:00, 634.43batch/s, loss=1.6928]


Epoch 19/20 completed. Average Loss: 1.6928


Epoch 20/20: 100%|██████████| 55/55 [00:00<00:00, 613.24batch/s, loss=1.6748]


Epoch 20/20 completed. Average Loss: 1.6748


,Class,Train F1 Score,Test F1 Score
0,CD14+ Monocytes,0.787415,0.791209
1,CD16+ Monocytes,0.000000,0.000000
2,CD20+ B cells,0.000000,0.000000
3,CD4+ T cells,0.554422,0.575221
4,CD8+ T cells,0.000000,0.000000
5,Megakaryocyte progenitors,0.000000,0.000000
6,Monocyte-derived dendritic cells,0.373563,0.459627
7,NK cells,0.000000,0.000000
8,NKT cells,0.000000,0.000000
9,Plasmacytoid dendritic cells,0.000000,0.000000


# compare results

In [11]:
t_dict = {'swav': {
            "dimensionality_reduction": ["pca"],
            "prot_decoding_loss_scaler": [0],
            "cvae_loss_scaler": [0.0],
            "k_neighbors": [7],
            "training_type": ['semi_supervised'],
            "fine_tuning_epochs": [150],
            "pretraining_epochs": [150],
            "augmentation_type": ['community'],
            'batch_size': [1024],
            # 'dataset_id':['hlca'],
            'model_name_version': [3]
        },
    
            'scpoli': {
            'latent_dims': [8],
            'batch_size': [1024],
            'experiment_name': ['scpoli-semi'],
            # 'dataset_id':['hlca'],
            'model_name_version': [3],
            }}

t = e.generate_trainers(t_dict)

Generating trainers for scpoli: 100%|██████████| 1/1 [00:00<00:00, 78.08it/s, Memory=3.1%, CPU=2.2%, trainer_name=scpoli_exp_scpoli-semi]


In [34]:
def evaluate_trainer(trainer, evaluate):
    trainer.setup()
    trainer.model = None
    # ref_emb = trainer.encode_ref()
    query_emb = trainer.encode_query()
    # ref_res = evaluate(ref_emb, st.ref.adata.obs)
    query_res = evaluate(query_emb, trainer.query.adata.obs['cell_type'])
    return query_res
    

In [38]:
def compare_trainers(trainers, evaluate):
    res_arr = [evaluate_trainer(trainer, evaluate) for trainer in trainers]
    keys = [trainer.name for trainer in trainers]

    # Create an empty DataFrame to hold concatenated results
    final_df = pd.DataFrame()
    
    # Iterate over DataFrames and keys to extract and concatenate the Test F1 Score
    for df, key in zip(res_arr, keys):
        # Extract the Test F1 Score column and rename it with the current key
        extracted_f1 = df[['Class', 'F1 Score']].copy()
        extracted_f1.rename(columns={'F1 Score': f'Test F1 {key}'}, inplace=True)
        
        # Merge or concatenate this extracted F1 score to the final DataFrame
        if final_df.empty:
            final_df = extracted_f1
        else:
            final_df = pd.merge(final_df, extracted_f1, on='Class', how='outer')

    return final_df

In [92]:
compare_trainers(t)

INFO - 10/16/24 16:23:08 - 0:00:00 - ============ Initialized logger ============
INFO - 10/16/24 16:23:08 - 0:00:00 - all_latent: None
                                     augmentation_type: community
                                     base_lr: 4.8
                                     batch_size: 1024
                                     cell_type_key: cell_type
                                     checkpoint_freq: 25
                                     condition_key: study
                                     crops_for_assign: [0, 1]
                                     cvae_loss_scaler: 0.0
                                     cvae_reg: 0
                                     dataset: pbmc-immune
                                     dataset_id: pbmc-immune
                                     debug: True
                                     default_values: {'dataset_id': 'pbmc-immune', 'model_name_version': 3, 'num_prototypes': 128, 'hidden_dim': 64, 'latent_dims': 8, 'batch_size'

Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 

Embedding dictionary:
 	Num conditions: [5]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



Epoch 20/20: 100%|██████████| 55/55 [00:00<00:00, 607.77batch/s, loss=1.2073]


Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 

Embedding dictionary:
 	Num conditions: [5]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



Epoch 20/20: 100%|██████████| 55/55 [00:00<00:00, 653.85batch/s, loss=1.7352]


,Class,Test F1 swav_aug_comm_DR_pca_kN_7,Test F1 scpoli_exp_scpoli-semi
0,CD14+ Monocytes,0.855967,0.876106
1,CD16+ Monocytes,0.000000,0.000000
2,CD20+ B cells,0.844156,0.022222
3,CD4+ T cells,0.785498,0.544503
4,CD8+ T cells,0.000000,0.000000
5,Macro Avg,0.460282,0.182745
6,Megakaryocyte progenitors,0.000000,0.000000
7,Micro Avg,0.707094,0.446224
8,Monocyte-derived dendritic cells,0.695652,0.384615
9,NK cells,0.770833,0.000000


# add knn results

In [28]:
import interpretable_ssl.evaluation.knn
importlib.reload(interpretable_ssl.evaluation.knn)

from interpretable_ssl.evaluation.knn import *


In [29]:
tr = t[0]
tr

In [31]:
tr.setup()
tr.model = None
# ref_emb = trainer.encode_ref()
query_emb = tr.encode_query()
    

INFO - 10/20/24 12:35:31 - 0:00:00 - ============ Initialized logger ============
INFO - 10/20/24 12:35:31 - 0:00:00 - all_latent: None
                                     augmentation_type: community
                                     base_lr: 4.8
                                     batch_size: 1024
                                     cell_type_key: cell_type
                                     checkpoint_freq: 25
                                     condition_key: study
                                     crops_for_assign: [0, 1]
                                     cvae_loss_scaler: 0.0
                                     cvae_reg: 0
                                     dataset: pbmc-immune
                                     dataset_id: pbmc-immune
                                     debug: True
                                     default_values: {'dataset_id': 'pbmc-immune', 'model_name_version': 3, 'num_prototypes': 128, 'hidden_dim': 64, 'latent_dims': 8, 'batch_size'

Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 

Embedding dictionary:
 	Num conditions: [5]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



100%|██████████| 5/5 [00:00<00:00,  8.87it/s]


In [32]:
knn_classifier_with_f1_report(query_emb, tr.query.adata.obs['cell_type'])

,Class,F1 Score
0,CD14+ Monocytes,0.935622
1,CD16+ Monocytes,0.720930
2,CD20+ B cells,0.971429
3,CD4+ T cells,0.848586
4,CD8+ T cells,0.000000
5,Megakaryocyte progenitors,1.000000
6,Monocyte-derived dendritic cells,0.905028
7,NK cells,0.777143
8,NKT cells,0.636943
9,Plasmacytoid dendritic cells,1.000000


In [39]:
compare_trainers(t, knn_classifier_with_f1_report)

INFO - 10/20/24 12:37:51 - 0:00:00 - ============ Initialized logger ============
INFO - 10/20/24 12:37:51 - 0:00:00 - all_latent: None
                                     augmentation_type: community
                                     base_lr: 4.8
                                     batch_size: 1024
                                     cell_type_key: cell_type
                                     checkpoint_freq: 25
                                     condition_key: study
                                     crops_for_assign: [0, 1]
                                     cvae_loss_scaler: 0.0
                                     cvae_reg: 0
                                     dataset: pbmc-immune
                                     dataset_id: pbmc-immune
                                     debug: True
                                     default_values: {'dataset_id': 'pbmc-immune', 'model_name_version': 3, 'num_prototypes': 128, 'hidden_dim': 64, 'latent_dims': 8, 'batch_size'

Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 

Embedding dictionary:
 	Num conditions: [5]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



100%|██████████| 5/5 [00:00<00:00, 13.37it/s]


Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 

Embedding dictionary:
 	Num conditions: [5]
 	Embedding dim: [10]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 10
	Mean/Var Layer in/out: 64 8
Decoder Architecture:
	First Layer in, out and cond:  8 64 10
	Output Layer in/out:  64 4000 



100%|██████████| 5/5 [00:00<00:00, 12.20it/s]


,Class,Test F1 swav_aug_comm_DR_pca_kN_7,Test F1 scpoli_exp_scpoli-semi
0,CD14+ Monocytes,0.935622,0.942222
1,CD16+ Monocytes,0.720930,0.675000
2,CD20+ B cells,0.971429,0.966292
3,CD4+ T cells,0.848586,0.813036
4,CD8+ T cells,0.000000,0.114286
5,Megakaryocyte progenitors,1.000000,0.857143
6,Monocyte-derived dendritic cells,0.905028,0.854271
7,NK cells,0.777143,0.688172
8,NKT cells,0.636943,0.434211
9,Plasmacytoid dendritic cells,1.000000,0.911765
